In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from typing import List, Optional
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from src.utils.file_utils import convert_time_to_seconds, create_full_date

In [3]:
conn = sqlite3.connect("data/athle_results.sqlite")
try:
    df = pd.read_sql_query(f"SELECT * FROM results", conn)
except Exception as e:
    print(f"Erreur lors de la lecture de la base : {e}")

In [104]:
conn.close()

In [98]:
conn = sqlite3.connect("data/athle_results.sqlite")
try:
    df = pd.read_sql_query(f"SELECT * FROM athletes", conn)
except Exception as e:
    print(f"Erreur lors de la lecture de la base : {e}")

In [99]:
df

,seq,name,club,sex
0,4851475245544851495050494554,VAUDOIS Aurelien,- Athle 92*,M
1,495046534653504950494257,DIRAT Franck,- Athle 92*,M
2,49504851445551484851475246535148,POTTIER Quentin,- Athle 92*,M


In [2]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
db_url = os.getenv("DB_URL")

# Crée l'engine SQLAlchemy
engine = create_engine(db_url)

# Exemple : lire la table results
df = pd.read_sql_query("SELECT * FROM results", engine)
print(df.head())

   id                           seq     club        date  \
0   1  4851475245544851495050494554  92111.0  2025-03-16   
1   2  4851475245544851495050494554  92111.0  2025-02-09   
2   3  4851475245544851495050494554  92111.0  2025-02-02   
3   4  4851475245544851495050494554  92111.0  2025-01-25   
4   5  4851475245544851495050494554  92111.0  2025-01-19   

                      epreuve     tour                 pl               perf  \
0                  5 Km Route  H.Stade  178 (163M - 86SE)  15'37'' (15'32'')   
1  Cross court masculin / sem    Cross                169            17'53''   
2           800m Piste Courte  Final 1                  3           1'54''38   
3           800m Piste Courte  Final 2                  2           1'55''84   
4           800m Piste Courte  Final 1                  6           1'55''51   

  perf_seconds vt  niv    pts            ville  annee  lieu  
0         None     IR3    0.0            Lille   2025  None  
1         None                    

In [8]:
df_1500_ava = df[(df.seq == "4851475245544851495050494554")& (df.epreuve.isin(['1 500m', '1 500m Piste Courte']))]
df_1500_ava.shape

(12, 15)

In [9]:
df_1500_ava

,id,seq,club,date,epreuve,tour,pl,perf,perf_seconds,vt,niv,pts,ville,annee,lieu
7,8,4851475245544851495050494554,92111.0,2024-07-20,1 500m,Final 1,12.0,3'56''08,None,,IR1,901.0,Ninove,2024,None
10,11,4851475245544851495050494554,92111.0,2024-07-03,1 500m,Final 2,1.0,3'53''37,None,,N4,934.0,Saint Maur Des Fosses,2024,None
16,17,4851475245544851495050494554,92111.0,2024-05-26,1 500m,Final 7,1.0,3'55''31,None,,IR1,911.0,Bruxelles,2024,None
32,33,4851475245544851495050494554,92111.0,2023-07-08,1 500m,Final 1,12,3'57''36,None,,IR1,886.0,Decines Charpieu,2023,None
42,43,4851475245544851495050494554,92111.0,2023-04-12,1 500m,Final 1,6,4'36''26,None,,R5,480.0,Le Plessis Robinson,2023,None
48,49,4851475245544851495050494554,21032.0,2022-07-06,1 500m,Final 5,1.0,3'55''75,None,,IR1,905.0,Saint Maur Des Fosses,2022,None
77,78,4851475245544851495050494554,21032.0,2019-07-06,1 500m,Final 1,2,4'20''61,None,,R2,0.0,Louhans,2019,None
111,112,4851475245544851495050494554,21032.0,2017-10-15,1 500m,Final 1,9,4'22''15,None,,R2,613.0,Montelimar,2017,None
113,114,4851475245544851495050494554,21032.0,2017-10-01,1 500m,Final 1,4,4'24''25,None,,R3,592.0,Dijon,2017,None
119,120,4851475245544851495050494554,21032.0,2017-05-21,1 500m,Final 1,8,4'26''98,None,,R3,566.0,Dole,2017,None


In [32]:
def get_results_from_db(seq):
    query = "SELECT * FROM results WHERE seq = %(seq)s"
    df = pd.read_sql_query(query, engine, params={"seq": seq})
    return df

df = get_results_from_db("49504851445551484851475246535148")

In [33]:
df

,id,seq,club,date,epreuve,tour,pl,perf,perf_seconds,vt,niv,pts,ville,annee,lieu
0,3329,49504851445551484851475246535148,,2021-10-30,Relais cross tcc - mixte,Relais 2,5.0,25'51'',None,,,,Pontcharra,2021,None
1,3326,49504851445551484851475246535148,,2021-12-18,Relais tc,Cross,8.0,28'42'',None,,,,Firminy,2021,None
2,3314,49504851445551484851475246535148,,2022-09-25,Relais Medley Long (1200-400-800-1600),Final 1,1,9'59''01,None,,,0.0,Grenoble,2022,None
3,3313,49504851445551484851475246535148,,2022-10-09,Ekiden,Relais 3,2 (2),2h13'36'',None,,,,Grenoble,2022,None
4,3310,49504851445551484851475246535148,,2022-12-04,Relais tc,Cross,9,23'35'',None,,,,Echirolles,2022,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,3280,49504851445551484851475246535148,92111.0,2024-12-15,3 000m Piste Courte,Final 1,3,8'36''63,None,,IR1,870.0,Lievin,2024,None
60,3279,49504851445551484851475246535148,92111.0,2025-01-19,1 500m Piste Courte,Final 1,3,4'02''27,None,,IR2,867.0,Nogent Sur Oise,2025,None
61,3278,49504851445551484851475246535148,92111.0,2025-01-25,1 500m Piste Courte,Final 1,10,4'02''07,None,,IR2,870.0,Lievin,2025,None
62,3277,49504851445551484851475246535148,92111.0,2025-02-09,Cross court masculin / sem,Cross,19,15'08'',None,,,,Lisses,2025,None


In [35]:
df[df.epreuve ==  '10 Km Route']

,id,seq,club,date,epreuve,tour,pl,perf,perf_seconds,vt,niv,pts,ville,annee,lieu
30,3311,49504851445551484851475246535148,38072.0,2022-11-20,10 Km Route,H.Stade,67 (67M - 49SE),31'35'' (31'31''),None,,IR1,,Venissieux,2022,None
45,3275,49504851445551484851475246535148,92043.0,2025-03-30,10 Km Route,H.Stade,131 (130M - 85SE),32'23'' (32'18''),None,,IR2,763.0,Valenciennes,2025,None


In [36]:
from dotenv import load_dotenv
load_dotenv()
db_url = os.getenv("DB_URL")
engine = create_engine(db_url)

In [53]:
from src.utils.http_utils import search_athletes

athletes = search_athletes('quentin pottier')
athletes

[{'hactseq': '27493039',
  'name': 'POTTIER Quentin',
  'club': ' - Reveil Sportif De St Cyr/loire',
  'sex': 'M',
  'seq': '49504455475242574851514848514257'},
 {'hactseq': '23703450',
  'name': 'POTTIER Quentin',
  'club': ' - Athle 92*',
  'sex': 'M',
  'seq': '49504851445551484851475246535148'}]

In [54]:
options = [f"{a['name']} ({a['club']})" for a in athletes]


In [55]:
options

['POTTIER Quentin ( - Reveil Sportif De St Cyr/loire)',
 'POTTIER Quentin ( - Athle 92*)']

In [56]:
options.index('POTTIER Quentin ( - Athle 92*)')

1

In [58]:
selected = athletes[options.index('POTTIER Quentin ( - Athle 92*)')]
selected

{'hactseq': '23703450',
 'name': 'POTTIER Quentin',
 'club': ' - Athle 92*',
 'sex': 'M',
 'seq': '49504851445551484851475246535148'}

In [59]:
if selected:
    seq = selected['seq']
    name = selected['name']
    club = selected['club']
    sex = selected['sex']

In [60]:
seq

'49504851445551484851475246535148'

In [61]:
def get_results_from_db(seq):
    query = "SELECT * FROM results WHERE seq = %(seq)s"
    df = pd.read_sql_query(query, engine, params={"seq": seq})
    return df

df = get_results_from_db(seq)
df.shape

(64, 15)

In [62]:
df.seq.unique()

array(['49504851445551484851475246535148'], dtype=object)